In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
import langchain_core, langchain_ollama
import pprint

#langchain_core : 0.3.10
#langchain_ollama : 0.2.0
print("langchain_core : " + langchain_core.__version__)
print("langchain_ollama : " + langchain_ollama.__version__)

# model="llama3.2:3b" 에서 했으나, tool calling 이 제대로 적용되지 않음.
# 버전은 낮지만, 더 큰 모델을 사용했더니 성공. llama3.1:8b
llm = ChatOllama(
    model="gemma2:2b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
)

langchain_core : 0.3.10
langchain_ollama : 0.2.0


In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("container", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://sourcebox.dev/blog/20241014/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


5209

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=100, 
    add_start_index=True
)

all_splits = text_splitter.split_documents(docs)

In [19]:
len(all_splits[0].page_content)

340

In [20]:
len(all_splits)

15

In [21]:
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OllamaEmbeddings(
        model="gemma2:2b",
        base_url="http://host.docker.internal:11434"
    )
)       

In [32]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":4}
)

In [33]:
retrieved_docs = retriever.invoke(
    "Tool Calling"
)

In [34]:
len(retrieved_docs)

4

In [42]:
print(retrieved_docs[2])

page_content='#langchain_core : 0.3.10
#langchain_ollama : 0.2.0
print("langchain_core : " + langchain_core.__version__)
print("langchain_ollama : " + langchain_ollama.__version__)
AI 모델로 llama3.1:8b 버전을 선택했다. 원래는 3b 모델을 사용해보려 했으나, 툴 호출 기능이 제대로 작동하지 않아, 8b모델로 했더니 결과가 만족스러웠다. 참고로 gemma2 는 tool calling 기능이 없었다.
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0,
    base_url="http://host.docker.internal:11434"
)
기본적인 수학 계산을 AI에게 요청해 봤다.
ai_msg = llm.invoke(input="What is 3 * 12? Also, what is 11 + 49?")
이정도 질의는 별다른 셋팅이 없어도 정확하게 잘 나온다. tool-calling 기능이 필요하지 않은 수준이다.
print(ai_msg.content)

"""
Two simple math questions!

**1. 3 * 12 = 36**

Multiplying 3 by 12 gives us a product of 36.

**2. 11 + 49 = 60**' metadata={'source': 'https://sourcebox.dev/blog/20241014/', 'start_index': 729}
